In [6]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow.keras import regularizers
import tensorflow as tf

In [7]:
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

In [8]:
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        scaler = StandardScaler()
        data = scaler.fit_transform(data)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

In [9]:
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial_Signals/'
	# load all 9 files as a single array
	filenames = list()
	# body acceleration
	#filenames += ['acc_x.txt', 'acc_y.txt', 'acc_z.txt']
	# body gyroscope
	filenames += ['gyr_x.txt', 'gyr_y.txt', 'gyr_z.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

In [10]:
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'Dataset1/')
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'Dataset1/')
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	return trainX, trainy, testX, testy

In [11]:
trainX, trainy, testX, testy=load_dataset()

In [12]:
trainX.shape

(33104, 128, 3)

In [13]:
testX.shape

(3740, 128, 3)

In [14]:
save('trainX.npy', trainX)
save('trainY.npy', trainy)
save('testX.npy', testX)
save('testY.npy', testy)

In [15]:
X_train = load('trainX.npy')
X_test = load('testX.npy')
y_train = load('trainy.npy')
y_test = load('testy.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(33104, 128, 3) (33104, 118) (3740, 128, 3) (3740, 118)


In [16]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [17]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [18]:
model1 = Sequential()
model1.add(LSTM(500, input_shape=(n_timesteps,n_features)))
model1.add(Dense(340, activation='relu'))
model1.add(Dropout(0.1))
model1.add(Dense(100, activation='relu'))
model1.add(Dense(n_outputs, activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 500)               1008000   
                                                                 
 dense (Dense)               (None, 340)               170340    
                                                                 
 dropout (Dropout)           (None, 340)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               34100     
                                                                 
 dense_2 (Dense)             (None, 118)               11918     
                                                                 
Total params: 1,224,358
Trainable params: 1,224,358
Non-trainable params: 0
_________________________________________________________________


In [19]:
history = model1.fit(X_training, y_training, epochs=10, verbose=False, validation_data=(X_validation, y_validation), batch_size=100)

In [20]:
loss, accuracy = model1.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model1.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9112
Testing Accuracy: 0.8487
